# 🤖 Entraînement YOLO pour Invoice ML System

Ce notebook vous permet d'entraîner votre modèle YOLO sur Google Colab avec GPU gratuit.

## 📋 Prérequis

1. Avoir annoté au moins 100 factures dans Label Studio
2. Avoir exporté les annotations (voir README.md)
3. Avoir préparé le dataset YOLO

## 🚀 Instructions

1. Activer le GPU: Runtime > Change runtime type > GPU
2. Uploader votre dataset (fichier zip du dossier yolo_dataset)
3. Exécuter les cellules dans l'ordre
4. Télécharger le modèle entraîné à la fin

## 1️⃣ Configuration de l'environnement

In [ ]:
# Vérifier le GPU
!nvidia-smi

In [ ]:
# Installer les dépendances
!pip install ultralytics==8.0.228 -q

## 2️⃣ Upload du dataset

**Option A: Upload manuel**

Utilisez le panneau de gauche pour uploader votre fichier `yolo_dataset.zip`

**Option B: Depuis Google Drive**

Décommentez et exécutez la cellule suivante:

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !cp /content/drive/MyDrive/yolo_dataset.zip .

In [ ]:
# Décompresser le dataset
!unzip -q yolo_dataset.zip
!ls -la yolo_dataset/

## 3️⃣ Configuration de l'entraînement

In [ ]:
from ultralytics import YOLO
import torch

# Paramètres
MODEL = 'yolov8n.pt'  # n=nano (rapide), s=small, m=medium, l=large
EPOCHS = 100
BATCH = 16
IMGSZ = 640
DATA_YAML = 'yolo_dataset/data.yaml'

print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
print(f"Modèle: {MODEL}")
print(f"Epochs: {EPOCHS}")

## 4️⃣ Entraînement du modèle

In [ ]:
# Charger le modèle pré-entraîné
model = YOLO(MODEL)

# Lancer l'entraînement
results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    batch=BATCH,
    imgsz=IMGSZ,
    patience=20,  # Early stopping
    save=True,
    plots=True,
    device=0,  # GPU
    project='invoice_training',
    name='yolo_run',
    verbose=True
)

## 5️⃣ Évaluation

In [ ]:
# Évaluer sur le set de test
metrics = model.val(split='test')

print(f"\n📊 Résultats:")
print(f"   Precision: {metrics.box.p.mean():.3f}")
print(f"   Recall:    {metrics.box.r.mean():.3f}")
print(f"   mAP@0.5:   {metrics.box.map50:.3f}")
print(f"   mAP@0.5:0.95: {metrics.box.map:.3f}")

## 6️⃣ Visualisation des résultats

In [ ]:
# Afficher les courbes d'entraînement
from IPython.display import Image, display

display(Image(filename='invoice_training/yolo_run/results.png'))
display(Image(filename='invoice_training/yolo_run/confusion_matrix.png'))

## 7️⃣ Test sur une image

In [ ]:
# Charger le meilleur modèle
best_model = YOLO('invoice_training/yolo_run/weights/best.pt')

# Tester sur une image
test_image = 'yolo_dataset/test/images/test_image.jpg'  # Remplacer par votre image
results = best_model(test_image)

# Afficher les résultats
results[0].show()

## 8️⃣ Télécharger le modèle

In [ ]:
# Zipper le modèle et les résultats
!zip -r invoice_model.zip invoice_training/yolo_run/weights/

# Télécharger
from google.colab import files
files.download('invoice_model.zip')

print("\n✅ Téléchargement lancé!")
print("\n📝 Prochaines étapes:")
print("   1. Extraire le modèle dans data/models/")
print("   2. Lancer l'API: python api/app.py")
print("   3. Tester: python scripts/test_api.py")

## 🎉 Terminé!

Votre modèle est maintenant entraîné. Téléchargez-le et utilisez-le dans votre projet.

### Conseils pour améliorer les performances:

1. **Annotez plus de données** (200-300 factures)
2. **Utilisez un modèle plus grand** (yolov8s ou yolov8m)
3. **Augmentez les epochs** (150-200)
4. **Ajustez les hyperparamètres**

### Ressources:

- [YOLO Documentation](https://docs.ultralytics.com/)
- [Guide d'entraînement](https://docs.ultralytics.com/modes/train/)
- [Hyperparameter tuning](https://docs.ultralytics.com/guides/hyperparameter-tuning/)